In [1]:
# TDO COMBINE TRAIN HISTORY DATA TO ONE FILE PER DAY
# TODO CALC TRAVALED PASSENGERN FROM TO FOR EACH DAY FOR EVERY STATION TO ONE! NEXT STATION

In [2]:
import os
import pandas as pd
from pathlib import Path
from datetime import datetime



cwd = os.getcwd()
dep_table_path = os.path.join(cwd, 'generated/departure_tables_raw')
dep_table_path

'/home/jovyan/work/generated/departure_tables_raw'

In [3]:
raw_departure_tables = []
departure_tables_dates = []
departure_tables_dates_ts = []
# LOAD ALL CSV FILES INTO ONE BIG CHUNK
files = [ f for f in os.listdir( dep_table_path)] #list comprehension version.

for f in files:
    if(Path(f).suffix == ".csv"):
        filename = str(Path(f).stem)
        if '_' in filename and len(filename.split('_')) == 4:
            # THE GOAL IS TO COMBINE ALL TRAIN DEPARTURES FROM THE SAME DAY
            # SO THE FILE TIMESTAMP IN ITS NAME CORRESPDS TO THE DATE OF RECORD = TRAIN DEPARTURES
            # BY ADDING THE NEW DATE TO THE PD DF ITS IS POSSIBLE TO SORT FOR DATES LATER ON
            timestamp = int(filename.split('_')[3])
            date = datetime.fromtimestamp(timestamp)
            # THIS WILL BE ADDED TO THE DF AS NEW COLUMN AFTER LOADING
            combined_day = str(date.day).zfill(2) + str(date.month).zfill(2) + str(date.year).zfill(2)
            departure_tables_dates.append(combined_day)
            departure_tables_dates_ts.append(timestamp)
            # LOAD FILE TO DF
            abs_path = os.path.join(dep_table_path,f)
            df = pd.read_csv(abs_path,sep=';',encoding="utf-8",dtype = str)
            
            # ADD NEW COLUMNS
            df['date'] = pd.Series([date for x in range(len(df.index))])
            df['combined_date'] = pd.Series([combined_day for x in range(len(df.index))])
            
            raw_departure_tables.append(df)


            
# CREATE DATASET FROM DATE VALUES
df_departure_tables_dates = pd.DataFrame(data={'recorded_days':departure_tables_dates})
            
# AFER LOADING ALL RAW FILE IN CREATE ONE GIANT DATASET
departures_combined = pd.concat(raw_departure_tables)
departures_combined.head(5)


,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date
0,2021-11-29T21:27:00.000Z,Bayreuth Hbf,RE,RE 30,30,3094,8000028,"8000028%2021-11-29T21:34:00.000Z%7,8001348,800...",602,2021-11-29 21:38:43,29112021
1,2021-11-29T22:57:00.000Z,Bayreuth Hbf,RE,RE 30,30,3099,8000028,"8000028%2021-11-29T22:58:00.000Z%1,8000974,800...",602,2021-11-29 21:38:43,29112021
2,2021-11-30T04:00:00.000Z,Bayreuth Hbf,RE,RE 30,30,3400,8000028,"8000028%2021-11-30T04:00:00.000Z%,8001348,8004...",602,2021-11-29 21:38:43,29112021
3,2021-11-30T04:45:00.000Z,Bayreuth Hbf,RE,RE 38,38,59318,8000028,"8000028%2021-11-30T04:45:00.000Z%,8000974,8004...",602,2021-11-29 21:38:43,29112021
4,2021-11-30T04:51:00.000Z,Bayreuth Hbf,RE,RE 30,30,3450,8000028,"8000028%2021-11-30T04:51:00.000Z%,8001348,8000...",602,2021-11-29 21:38:43,29112021


In [4]:
# REMOVE DUPLICATE TRAINS TRAIN_ID AND START_STATION
departures_combined = departures_combined.drop_duplicates(['number','current_station_name','combined_date'],keep= 'last')
departures_combined.head(5)


,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date
0,2021-11-29T21:27:00.000Z,Bayreuth Hbf,RE,RE 30,30,3094,8000028,"8000028%2021-11-29T21:34:00.000Z%7,8001348,800...",602,2021-11-29 21:38:43,29112021
15,2021-11-29T21:28:00.000Z,Bielefeld-Brackwede,RB,RB 69,69,90085,8000048,"8000048%2021-11-29T21:33:00.000Z%5,8000036,",426,2021-11-29 21:38:43,29112021
16,2021-11-29T21:44:00.000Z,Bielefeld-Brackwede,RB,RB 67,67,79316,8000048,"8000048%2021-11-29T21:44:00.000Z%0,8000036,",426,2021-11-29 21:38:43,29112021
124,2021-11-29T21:45:00.000Z,Landshut(Bay)Hbf,RB,RB 45,45,27139,8000217,"8000217%2021-11-29T21:45:00.000Z%0,8003514,800...",426,2021-11-29 21:38:43,29112021
125,2021-11-29T21:48:00.000Z,Landshut(Bay)Hbf,RE,RE 3,3,4089,8000217,"8000217%2021-11-29T21:49:00.000Z%1,8004084,800...",602,2021-11-29 21:38:43,29112021


In [5]:
# LETS TEST THE DATA
# GET ALL TRAIN DEPARTURES FROM Bayreuth Hbf [8000028] @ 29.11.2021
# RV + FV + DB ONLY
tmp = departures_combined.loc[departures_combined['current_station'] == str(8000028)]
tmp = tmp.loc[tmp['combined_date'] == departure_tables_dates[0]]
tmp = tmp.reset_index()
tmp.head(100)

,index,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date
0,0,2021-11-29T21:27:00.000Z,Bayreuth Hbf,RE,RE 30,30,3094,8000028,"8000028%2021-11-29T21:34:00.000Z%7,8001348,800...",602,2021-11-29 21:38:43,29112021
1,0,2021-11-29T22:57:00.000Z,Bayreuth Hbf,RE,RE 30,30,3099,8000028,"8000028%2021-11-29T22:59:00.000Z%2,8000974,800...",602,2021-11-29 22:23:53,29112021
2,0,2021-11-30T04:00:00.000Z,Bayreuth Hbf,RE,RE 30,30,3400,8000028,"8000028%2021-11-30T04:00:00.000Z%,8001348,8004...",602,2021-11-29 23:41:55,29112021
3,1,2021-11-30T04:45:00.000Z,Bayreuth Hbf,RE,RE 38,38,59318,8000028,"8000028%2021-11-30T04:45:00.000Z%,8000974,8004...",602,2021-11-29 23:41:55,29112021
4,2,2021-11-30T04:51:00.000Z,Bayreuth Hbf,RE,RE 30,30,3450,8000028,"8000028%2021-11-30T04:51:00.000Z%,8001348,8000...",602,2021-11-29 23:41:55,29112021
5,3,2021-11-30T05:24:00.000Z,Bayreuth Hbf,RE,RE 30,30,3402,8000028,"8000028%2021-11-30T05:24:00.000Z%,8004759,8004...",602,2021-11-29 23:41:55,29112021
6,4,2021-11-30T06:01:00.000Z,Bayreuth Hbf,RE,RE 32,32,59300,8000028,"8000028%2021-11-30T06:01:00.000Z%,8005895,8000...",602,2021-11-29 23:41:55,29112021
7,5,2021-11-30T06:03:00.000Z,Bayreuth Hbf,RE,RE 32,32,3454,8000028,"8000028%2021-11-30T06:03:00.000Z%,8004759,8004...",602,2021-11-29 23:41:55,29112021
8,6,2021-11-30T06:12:00.000Z,Bayreuth Hbf,RE,RE 30,30,3404,8000028,"8000028%2021-11-30T06:12:00.000Z%,8001348,8004...",602,2021-11-29 23:41:55,29112021
9,7,2021-11-30T06:32:00.000Z,Bayreuth Hbf,RE,RE 30,30,3081,8000028,"8000028%2021-11-30T06:32:00.000Z%,8004126,8002...",602,2021-11-29 23:41:55,29112021


In [6]:
# COUNT THE UNIQUE TRAINS
#THE TRAIN_ID IS UNIQUE FOR EACH TRAIN AND IDENTIFIES THE TRAIN OVER ITS COMPLETE COURSE
# ONE UNIQUE ID = ONE ZUGLAUF
# FROM START TO END-STATION
tmp = departures_combined.loc[departures_combined['combined_date'] == departure_tables_dates[0]]
tmp['number'].nunique()

6793

In [7]:
# REMOVE DUPLICATES IN THE DATES
df_departure_tables_dates = df_departure_tables_dates.drop_duplicates()

In [8]:
# SAVE RESULTS
departures_combined.to_csv("./generated/4_departure_table.csv",sep=';',encoding="utf-8",index=False)
df_departure_tables_dates.to_csv("./generated/4_departure_table_day_list.csv",sep=';',encoding="utf-8",index=False)

In [9]:
# ALSO SAVE RESULTS FOR VISUALISATION
departures_combined.to_csv("./generated/visualisation_files/departure_table.csv",sep=';',encoding="utf-8",index=False)
df_departure_tables_dates.to_csv("./generated/visualisation_files/departure_table_day_list.csv",sep=';',encoding="utf-8",index=False)